In [69]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS,Galactic,FK4,FK5
from astropy.coordinates import Angle,Latitude,Longitude
from astropy import units as u

In [15]:
f1=pd.read_csv("cosmos.csv")
f2=pd.read_csv("alhambra.csv")

In [16]:
f1

,id_cosmos,ra,dec,lp_type
0,13,149.556959,1.391558,0
1,101,149.612364,1.391676,0
2,118,150.646288,1.391693,0
3,222,149.777400,1.391864,0
4,240,149.224797,1.391768,0
...,...,...,...,...
1048561,759333,150.065904,2.147631,0
1048562,759340,150.985652,2.147398,0
1048563,759349,150.781864,2.147475,0
1048564,759358,150.533326,2.147606,0


In [17]:
f2

,id,ra,dec
0,81441100124,150.3341,2.5676
1,81441100133,150.4740,2.5668
2,81441100140,150.2687,2.5682
3,81441100152,150.2530,2.5684
4,81441100153,150.3063,2.5683
...,...,...,...
38422,81441411438,150.3511,1.8429
38423,81441411443,150.3401,1.8428
38424,81441411455,150.3051,1.8423
38425,81441411466,150.2914,1.8419


identifying similar objects in both catalogs with max seperation of 3 arcseconds 

In [18]:
ra1=f1["ra"]
dec1=f1['dec']
ra2=f2["ra"]
dec2=f2['dec']


c = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)
catalog = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)
idx, d2d, d3d = c.match_to_catalog_sky(catalog)


max_sep = 3 * u.arcsec
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]


cosmos_new=pd.DataFrame(data=c_matches)
cosmos_new.to_csv("cosmos_new.csv",index=False)
alhambra_new=pd.DataFrame(data=catalog_matches)
alhambra_new.to_csv("alhambra_new.csv",index=False)



In [ ]:
cosmos_new=pd.read_csv("cosmos_new.csv")

In [24]:
cosmos_new[['ra','dec']] = cosmos_new.coor.str.split(",",expand=True,)
cosmos_new.drop("coor",axis=1,inplace=True)
cosmos_new['ra'] = cosmos_new['ra'].str.replace("\n",'')
cosmos_new.to_csv("cosmos_new.csv",index=False)

In [33]:
alhambra_new=pd.read_csv("alhambra_new.csv")
alhambra_new[['ra','dec']] = alhambra_new.coor.str.split(",",expand=True,)
alhambra_new.drop("coor",axis=1,inplace=True)
alhambra_new['ra'] = alhambra_new['ra'].str.replace("\n",'')
alhambra_new.to_csv("alhambra_new.csv",index=False)

In [51]:
cosmos=pd.read_csv('cosmos.csv')
alhambra=pd.read_csv("alhambra.csv")
cosmos_new=pd.read_csv("cosmos_new.csv")
alhambra_new=pd.read_csv("alhambra_new.csv")

lp_type is true classification: galaxy: 0 and star: 1

In [52]:
cosmos

,id_cosmos,ra,dec,lp_type
0,13,149.556959,1.391558,0
1,101,149.612364,1.391676,0
2,118,150.646288,1.391693,0
3,222,149.777400,1.391864,0
4,240,149.224797,1.391768,0
...,...,...,...,...
1048561,759333,150.065904,2.147631,0
1048562,759340,150.985652,2.147398,0
1048563,759349,150.781864,2.147475,0
1048564,759358,150.533326,2.147606,0


In [53]:
alhambra

,id,ra,dec
0,81441100124,150.3341,2.5676
1,81441100133,150.4740,2.5668
2,81441100140,150.2687,2.5682
3,81441100152,150.2530,2.5684
4,81441100153,150.3063,2.5683
...,...,...,...
38422,81441411438,150.3511,1.8429
38423,81441411443,150.3401,1.8428
38424,81441411455,150.3051,1.8423
38425,81441411466,150.2914,1.8419


In [54]:
alhambra_new

,ra,dec
0,150.2965,1.8450
1,150.4716,1.8449
2,150.2738,1.8449
3,149.9269,1.8485
4,149.9458,1.8469
...,...,...
35676,150.3423,2.0862
35677,150.3106,2.0865
35678,150.4612,2.0857
35679,149.8204,2.0866


In [55]:
cosmos_new

,ra,dec
0,150.296536,1.844926
1,150.471568,1.844804
2,150.273227,1.845281
3,149.926313,1.847976
4,149.945687,1.846862
...,...,...
35676,150.342347,2.086153
35677,150.311269,2.086832
35678,150.461230,2.085633
35679,149.820508,2.087352


In [58]:
alham=alhambra_new.merge(alhambra, how='left', on=['ra','dec'], sort=False)
cosm=cosmos_new.merge(cosmos_new.merge(cosmos, how='left', on=['ra','dec'], sort=False))

alham.drop(['ra','dec'],axis=1,inplace=True)
cosm.drop(['ra','dec'],axis=1,inplace=True)

class_change=pd.concat([alham,cosm],axis=1)

In [63]:
class_change.drop(class_change[class_change["lp_type"]==-99].index,inplace=True) # dropping data for which classification is not given
class_change.drop("id_cosmos",axis=1,inplace=True)

class_change.to_csv("class_change.csv",index=False)

In [203]:
class_change=pd.read_csv("class_change.csv")


In [207]:
raw_data=pd.read_csv("raw_data_final.csv")
raw_data=raw_data[raw_data["field"]==4]

len(raw_data['id'].unique())

38427

now dropping the duplicate values. these are objects in alhambra which matches with more than one object in cosmos

In [208]:
class_change.drop_duplicates(inplace=True)

In [209]:
len(class_change)

27156

In [210]:
len(class_change['id'].unique())

26747

In [213]:
class_change.drop(class_change[(class_change['id'].duplicated())&((class_change['lp_type']==0)|(class_change['lp_type']==-1)|(class_change['lp_type']==1)|(class_change['lp_type']=='NaN')|(class_change['lp_type']==2))].index,inplace=True)
class_change.drop(class_change[class_change['lp_type']==2].index,inplace=True)

len(class_change)

26505

In [216]:
new_raw=raw_data.merge(class_change,how='left',on='id')

new_raw

,id,field,fwhm,stellar_sextractor,f365w,f396w,f427w,f458w,f489w,f520w,...,f892w,f923w,f954w,j,h,ks,f814w,stellar_flag,class,lp_type
0,81441100119,4,11.91,0.03,20.664,20.592,20.275,19.825,19.476,19.304,...,17.748,17.483,17.661,17.205,16.812,16.590,17.963,0.00,0.0,NaN
1,81441100130,4,17.02,0.73,99.000,99.000,24.077,24.312,23.448,23.644,...,22.197,22.564,21.959,21.946,22.067,99.000,22.441,0.00,0.0,1.0
2,81441100134,4,6.39,0.03,99.000,24.022,22.285,21.932,21.456,21.238,...,18.855,18.702,18.838,18.451,18.057,17.845,19.163,0.16,0.0,0.0
3,81441100147,4,8.07,0.52,99.000,99.000,99.000,99.000,26.223,99.000,...,24.362,-99.000,24.907,24.796,23.356,99.000,25.035,0.50,-1.0,1.0
4,81441100151,4,8.29,0.69,99.000,24.568,25.246,99.000,25.053,25.063,...,99.000,-99.000,24.430,23.812,99.000,23.070,24.144,0.50,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38422,81441411447,4,12.20,0.01,23.590,23.456,23.774,23.905,23.921,23.899,...,22.638,22.605,23.433,22.730,23.027,22.176,23.102,0.50,0.0,0.0
38423,81441411449,4,18.64,0.03,99.000,25.356,99.000,25.045,25.143,25.875,...,22.966,22.340,23.262,21.748,21.362,20.605,22.913,0.50,-1.0,0.0
38424,81441411451,4,5.23,0.45,99.000,99.000,27.601,26.721,27.410,26.573,...,99.000,25.136,25.539,99.000,99.000,99.000,26.099,0.50,-1.0,0.0
38425,81441411453,4,5.20,0.84,23.305,22.768,22.240,21.925,21.759,21.673,...,20.538,20.542,20.446,20.400,20.208,20.584,20.713,0.89,1.0,NaN


In [217]:
new_raw.to_csv("new_raw.csv",index=False)

now here i have used the final results of sextractor of alhambra to remove errors in original classification as some objects are mismatched(may be due to change in no. of significant terms or that i have used d2d distance (ra and dec) in matching algo of astropy and not d3d distance (which consists of redshift as third coordinate). Therefore number of stars are reduced.

In [218]:
new_raw['class'] = new_raw['class'].astype(str).str.replace('-1','0')
new_raw['lp_type'].fillna(value=0,inplace=True)
new_raw['class'] = new_raw['class'].astype(float)
new_raw['lp_type'] = new_raw['lp_type'].astype(float)
new_raw['g/s']=new_raw['class']+new_raw['lp_type']
new_raw['g/s'] = new_raw['g/s'].astype(str).str.replace('2','1')
new_raw['g/s'] = new_raw['g/s'].astype(float)


In [224]:
len(new_raw[new_raw['g/s']==1])

2449

In [228]:
len(new_raw[(new_raw['g/s']==1)])

2449

In [234]:
len(new_raw[((new_raw["f814w"]<26)) & (new_raw["g/s"]==1)])

2448

In [235]:
new_raw.drop(["stellar_sextractor",'field','class','lp_type'],axis=1,inplace=True)
new_raw.to_csv("final_raw_to_use.csv")